In [50]:
import datetime
import os
import webbrowser
import pyautogui as pg
import time
import glob
import shutil
import random
import pandas as pd
today = datetime.date.today()
download_path = r"C:\Users\810202558\Downloads"
baseDownloadedDir = r'D:\PROJECTS\Appportal\downloaded'

In [51]:
def generate_month_list():
    today = datetime.date.today()
    current_month = today.month
    months_list = [str(month).zfill(2) for month in range(1, current_month + 1)]
    return months_list

def create_folder(folder_name):
    try:
        os.mkdir(os.path.join('downloaded',folder_name))
        print(f"Folder '{folder_name}' created successfully.")
    except FileExistsError:
        pass

def delfiles(dir):
    try:
        for filename in os.listdir(dir):
            file_path = os.path.join(dir, filename)
            if os.path.isfile(file_path):
                os.remove(file_path)
    except Exception as e:
        print(f"Error: {e}")

def sorted_files(source_dir):
    csv_files = glob.glob(os.path.join(source_dir,'*.csv'))
    today_csv_files = []
    for csv_file in csv_files:
        stat = os.stat(csv_file)
        file_date = datetime.date.fromtimestamp(stat.st_ctime)
        if file_date == today:
            today_csv_files.append(csv_file)
    sorted_files = sorted(
        today_csv_files, key=os.path.getctime, reverse=False)
    return sorted_files

def moving_files(sorted_files,val):
    for filename in sorted_files:
        namefile = filename.split("\\")[-1]
        dest_dir = os.path.join(baseDownloadedDir,val, namefile)
        shutil.move(filename, dest_dir)

# MPN 

In [ ]:
baseUrl = 'https://appportal.intranet.pajak.go.id/portal/download/lsnfjkasbnfjnasjkfnjbnjnjknbkjnfjknbjkfnbkjfnbi3939489184.php?p1='
# kpp = ['001']
kpp = ['001','002','003','004','005','006','007','008','009','097']
tahun = '2023'
jns_pajak = '411100000'
tgl_awal = '01'
tgl_akhir = '31'
bulan = generate_month_list()
# bulan = ['01']
valuta = ['1','2','3']
# valuta = ['3']

In [ ]:
webbrowser.open('https://appportal.intranet.pajak.go.id/login/')
time.sleep(2)
pg.hotkey('tab')
pg.typewrite('810202558', interval=0.1)
pg.hotkey('tab')
pg.typewrite('Gengsu!sh3r3', interval=0.1)
pg.hotkey('enter')

time.sleep(3)

for val in valuta:
    mpn_dir = os.path.join(baseDownloadedDir,val)
    #bikin folder
    if not os.path.exists(mpn_dir):
         
         os.makedirs(mpn_dir)
    for adm in kpp:
      for bln in bulan:
               url = baseUrl+adm+tahun+jns_pajak+tgl_awal+tgl_akhir+bln+val
               webbrowser.open_new_tab(url)
               if val =='1':
                   time.sleep(random.randint(2,8))
               else:
                   time.sleep(random.randint(1,3))   
               

    sorted_files = sorted_files(download_path)
    num_files = len(kpp)*len(bulan) #per valuta
    while len(sorted_files)<num_files:
        time.sleep(10)
        sorted_files = sorted_files(download_path)
    
    delfiles(mpn_dir)
    moving_files(sorted_files, val)

# MPN USD

In [ ]:
url_usd = f'https://appportal.intranet.pajak.go.id/portal/detilperekaman/exportkpp.php?xunit=110&xtahun=2023&xbulan1=01&xbulan2={bulan[-1]}'
webbrowser.open_new_tab(url_usd)
time.sleep(3)
shutil.move(os.path.join(download_path,'detildollar.xls'), r'D:\PROJECTS\Appportal\downloaded\2\detildollar.xls')

# SPM

In [ ]:
baseUrl = 'https://appportal.intranet.pajak.go.id/portal/spm/dataspmcsv.php?'
# kpp = ['001']
kpp = ['001','002','003','004','005','006','007','008','009','097']
tahun = '2023'
tgl_awal = '01'
tgl_akhir = '31'
bulan = generate_month_list()
# bulan = ['01']

In [ ]:
ant = '&'
for adm in kpp:
    for bln in bulan:
        url = baseUrl+'p1='+tgl_awal+ant+'p2='+tgl_akhir+ant+'p3='+bln+ant+'p4='+tahun+ant+'p5='+adm
        webbrowser.open_new_tab(url)   
        time.sleep(random.randint(1,3))
    
    spm_dir = os.path.join('D:\PROJECTS\Appportal\downloaded','spm')
    #bikin folder
    if not os.path.exists(spm_dir):
         os.makedirs(spm_dir)

    sorted_files = sorted_files(download_path)
    num_files = len(kpp)*len(bulan) #per valuta
    while len(sorted_files)<num_files:
        time.sleep(10)
        sorted_files = sorted_files(download_path)
    
    delfiles(spm_dir)
    moving_files(sorted_files, 'spm')

# ETL

In [16]:
idr_files = glob.glob(r'downloaded\1\*.csv')
usd_files = glob.glob(r'downloaded\2\*.csv')
dolar_file = glob.glob(r'downloaded\2\*.xls')
pbb_files = glob.glob(r'downloaded\3\*.csv')
spm_files = glob.glob(r'downloaded\spm\*.csv')

#### IDR

In [4]:
data_mpn = pd.DataFrame()
for file in idr_files:
    namefile = (file.split("\\")[-1].split("_")[-1])[:3]
    temp = pd.read_csv(file, dtype={'NPWP': 'str', 'KPP': 'str', 'CAB': 'str','PTNTP':'str',
                                            'PTMSPJ': 'str', 'TGLBYR': 'str', 'BLNBYR': 'str', 'THNBYR': 'str'})
    temp['ADMIN'] = namefile
    data_mpn = pd.concat([data_mpn, temp], axis=0, ignore_index=True)

data_mpn.rename(columns={'THNBYR': 'TAHUNBAYAR',
                    'BLNBYR': 'BULANBAYAR', 'TGLBYR': 'TANGGALBAYAR'}, inplace=True)
data_mpn['DATEBAYAR'] = data_mpn['TAHUNBAYAR']+data_mpn['BULANBAYAR']+data_mpn['TANGGALBAYAR']
data_mpn['DATEBAYAR'] = pd.to_datetime(data_mpn['DATEBAYAR'],format='%Y%m%d')
data_mpn['MASA_PAJAK'] = data_mpn['PTMSPJ'].str[0:2]
data_mpn['MASA_PAJAK2'] = data_mpn['PTMSPJ'].str[2:4]
data_mpn['TAHUN_PAJAK'] = data_mpn['PTMSPJ'].str[4:]
# data_mpn['DATEBAYAR'] = pd.to_datetime(data_mpn[['TANGGALBAYAR', 'BULANBAYAR', 'TAHUNBAYAR']])

#### USD

In [5]:
data_usd = pd.DataFrame()
for file in usd_files:
  namefile = (file.split("\\")[-1].split("_")[-1])[:3]
  temp = pd.read_csv(file, dtype={'NPWP': 'str', 'KPP': 'str', 'CAB': 'str','PTNTP':'str',
                                    'PTMSPJ': 'str', 'TGLBYR': 'str', 'BLNBYR': 'str', 'THNBYR': 'str'})
  data_usd['ADMIN'] = namefile
  data_usd = pd.concat([data_usd, temp], axis=0, ignore_index=True)
data_usd['NPWP'] = data_usd['NPWP'].str.strip()
data_usd['KPP'] = data_usd['KPP'].str.strip()
data_usd['CAB'] = data_usd['CAB'].str.strip()
data_usd.rename(columns={'THNBYR': 'TAHUNBAYAR',
                    'BLNBYR': 'BULANBAYAR', 'TGLBYR': 'TANGGALBAYAR'}, inplace=True)
data_usd['MASA_PAJAK'] = data_usd['PTMSPJ'].str[0:2]
data_usd['MASA_PAJAK2'] = data_usd['PTMSPJ'].str[2:4]
data_usd['TAHUN_PAJAK'] = data_usd['PTMSPJ'].str[4:]
data_usd['DATEBAYAR'] = data_usd['TAHUNBAYAR']+data_usd['BULANBAYAR']+data_usd['TANGGALBAYAR']
data_usd['DATEBAYAR'] = pd.to_datetime(data_usd['DATEBAYAR'],format='%Y%m%d')

In [6]:
data_dollar = pd.read_excel(r'downloaded\2\detildollar.xls', engine='xlrd', usecols=['NTPN','JUMLAH RUPIAH'])

*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'


In [7]:
data_usd = data_usd.merge(data_dollar,left_on='PTNTP', right_on='NTPN', how='left')

In [8]:
data_usd = data_usd.drop(columns=['JUMLAH','NTPN'])
data_usd = data_usd.rename(columns={'JUMLAH RUPIAH':'JUMLAH'})

In [9]:
data_mpnall = pd.concat([data_mpn,data_usd],axis=0,ignore_index=False)

In [10]:
data_mpnall[['NOP','KET']] = data_mpnall[['NOP','KET']].fillna('-')

In [11]:
data_mpnall.columns = [x.lower() for x in data_mpnall.columns]

In [12]:
data_mpnall = data_mpnall.rename(columns={'cab':'cabang','kjs':'kdbyar','masa_pajak':'masa','tahun_pajak':'tahun','jumlah':'nominal','ptntp':'ntpn',
 'noskssp':'nosk','id':'billing','pembuat billing':'pembuat', 'masa_pajak2':'masa2'})
data_mpnall['nospm'] = ''
data_mpnall['source'] = 1
data_mpnall['extra'] = ''
data_mpnall['tipe'] = ''

In [13]:
data_mpnall = data_mpnall.rename(columns={'kdbyar':'kdbayar'})

In [14]:
data_mpnall = data_mpnall.drop(columns=['alamat','kd bank','kpp_penerima'])
data_mpnall = data_mpnall[['admin', 'npwp', 'kpp', 'cabang', 'nama', 'kdmap', 'kdbayar', 'masa',
       'tahun', 'tanggalbayar', 'bulanbayar', 'tahunbayar', 'datebayar',
       'nominal', 'ntpn', 'bank', 'nosk', 'nospm', 'tipe', 'source', 'extra',
       'billing', 'nop', 'pembuat', 'ket', 'masa2']]

#### SPM

In [8]:
kpp = ['001','002','003','004','005','006','007','008','009','097']
spm_files = 'downloaded\spm'
spm_files = sorted_files(spm_files)

In [15]:
spm_dir = os.path.join('D:\PROJECTS\Appportal\downloaded','spm')


In [42]:
csv_files = glob.glob(os.path.join(download_path,'*.csv'))
today_csv_files = []
for csv_file in csv_files:
    stat = os.stat(csv_file)
    file_date = datetime.date.fromtimestamp(stat.st_ctime)
    today_csv_files.append(csv_file)
sorted_files = sorted(
    today_csv_files, key=os.path.getctime, reverse=False)

In [53]:
kpp = ["001", "002", "003", "004", "005", "006", "007", "008", "009", "097"]
tahun = "2023"
jns_pajak = "411100000"
tgl_awal = "01"
tgl_akhir = "31"
bulan = generate_month_list()
# bulan = ['01']
valuta = ["1", "2", "3"]
spm_sorted = sorted_files(download_path)
num_files = len(kpp) * len(bulan)  # per va luta
while len(spm_sorted) < num_files:
    time.sleep(10)
    spm_sorted = sorted_files(download_path)


KeyboardInterrupt: 

In [37]:
csv_files = glob.glob(os.path.join(source_dir, "*.csv"))
    today_csv_files = []
for csv_file in csv_files:
    stat = os.stat(csv_file)
    file_date = datetime.date.fromtimestamp(stat.st_ctime)
    if file_date == today:
        today_csv_files.append(csv_file)
sorted_files = sorted(today_csv_files, key=os.path.getctime, reverse=False)

IndentationError: unexpected indent (2079215990.py, line 2)

In [25]:
spm_downloaded = glob.glob(os.path.join(download_path, "Data_SPM*.csv"))

In [54]:
csv_files = glob.glob(os.path.join(download_path,'*.csv'))

In [60]:
x = csv_files[0].split("\\")
y = '\\'.join(a for a in x[:-1])

In [61]:
y

'C:\\Users\\810202558\\Downloads'